In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
import pickle
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from scipy.stats import randint, uniform



In [ ]:
male_players_file_path = '/content/drive/My Drive/male_players.csv'

male_players_data = pd.read_csv(male_players_file_path)

<ipython-input-132-ffbea9d3f572>:3: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  male_players_data = pd.read_csv(male_players_file_path)


In [ ]:
male_players_data.head()


,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


# Data Preprocessing

In [ ]:
# Function To Drop Columns With A Low Correlation

In [ ]:
def drop_low_correlation_columns(overall_dataframe, target_column, threshold=0.5):
    numerical_data = overall_dataframe.select_dtypes(include=['number'])
    columns_to_drop = []

    for column in numerical_data.columns:
        if column != target_column:
            corr = numerical_data[column].corr(numerical_data[target_column])
            if abs(corr) < threshold or pd.isna(corr):
                columns_to_drop.append(column)

    overall_dataframe = overall_dataframe.drop(columns=columns_to_drop)
    return overall_dataframe

result = drop_low_correlation_columns(male_players_data, 'overall')
result.head()

,player_url,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,/player/158023/lionel-messi/150002,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,1987-06-24,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,/player/20801/c-ronaldo-dos-santos-aveiro/150002,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,1985-02-05,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,/player/9014/arjen-robben/150002,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,1984-01-23,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,/player/41236/zlatan-ibrahimovic/150002,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,275000.0,1981-10-03,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,/player/167495/manuel-neuer/150002,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,63500000.0,300000.0,1986-03-27,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [ ]:
# Function to perform one hot encoding on the preferred foot column

def one_hot_encode_preferred_foot(df):
    result = df.copy()
    result = pd.get_dummies(result, columns=['preferred_foot'], prefix='foot')
    foot_columns = [col for col in result.columns if col.startswith('foot_')]
    for col in foot_columns:
        result[col] = result[col].astype(int)
    return result

result = one_hot_encode_preferred_foot(result)

In [ ]:
result.head()

,player_url,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,...,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,foot_Left,foot_Right
0,/player/158023/lionel-messi/150002,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,1987-06-24,...,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png,1,0
1,/player/20801/c-ronaldo-dos-santos-aveiro/150002,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,1985-02-05,...,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png,0,1
2,/player/9014/arjen-robben/150002,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,1984-01-23,...,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png,1,0
3,/player/41236/zlatan-ibrahimovic/150002,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,275000.0,1981-10-03,...,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png,0,1
4,/player/167495/manuel-neuer/150002,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,63500000.0,300000.0,1986-03-27,...,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png,0,1


In [ ]:
# Function to Drop Columns With More than 30 Percent Null Values

def drop_high_null_columns(data, threshold=0.3):
    thresh_count = int((1 - threshold) * len(data))
    data_cleaned = data.dropna(axis=1, thresh=thresh_count)
    return data_cleaned

result = drop_high_null_columns(result, threshold=0.3)

In [ ]:
result.head()

,player_url,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,...,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,foot_Left,foot_Right
0,/player/158023/lionel-messi/150002,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,1987-06-24,...,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png,1,0
1,/player/20801/c-ronaldo-dos-santos-aveiro/150002,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,1985-02-05,...,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png,0,1
2,/player/9014/arjen-robben/150002,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,1984-01-23,...,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png,1,0
3,/player/41236/zlatan-ibrahimovic/150002,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,275000.0,1981-10-03,...,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png,0,1
4,/player/167495/manuel-neuer/150002,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,63500000.0,300000.0,1986-03-27,...,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png,0,1


In [ ]:
# Dropping all categorical columns
def drop_categorical_columns(data):
    categorical_columns = data.select_dtypes(include=['object']).columns
    data = data.drop(columns=categorical_columns)
    return data

result = drop_categorical_columns(result)


Imputing

In [ ]:
# Function to insert the median value where there are Null values
def median_imputing(dataframes):
  for col in dataframes.columns:
    column_median = dataframes[col].median(skipna=True)
    dataframes[col] = dataframes[col].fillna(column_median)
  return dataframes

result = median_imputing(result)


In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   overall              161583 non-null  int64  
 1   potential            161583 non-null  int64  
 2   value_eur            161583 non-null  float64
 3   wage_eur             161583 non-null  float64
 4   passing              161583 non-null  float64
 5   dribbling            161583 non-null  float64
 6   physic               161583 non-null  float64
 7   movement_reactions   161583 non-null  int64  
 8   mentality_composure  161583 non-null  float64
 9   foot_Left            161583 non-null  int64  
 10  foot_Right           161583 non-null  int64  
dtypes: float64(6), int64(5)
memory usage: 13.6 MB


# Feature Engineering

In [ ]:
# I have already dropped the columns with low correlation, so it is just left with scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

def scale_independent_variables(dataframe, target_column):
    df_scaled = dataframe.copy()
    columns_to_scale = [col for col in df_scaled.columns if col != target_column]
    scaler = StandardScaler()
    df_scaled[columns_to_scale] = scaler.fit_transform(df_scaled[columns_to_scale])

    return df_scaled, scaler

scaled_df, scaler = scale_independent_variables(result, 'overall')
result = scaled_df


#Training The model

In [ ]:
# Separating our features from our labels
X = result.drop("overall",axis=1) # Features
y = result["overall"] # Labels



In [ ]:
# Splitting our data into training and testing modules
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)
X_train.head()

,potential,value_eur,wage_eur,passing,dribbling,physic,movement_reactions,mentality_composure,foot_Left,foot_Right
41592,-0.757937,-0.281638,-0.356368,0.095424,-0.018935,-0.538380,1.136656,0.085565,-0.553293,0.553293
29372,-0.278635,-0.344450,-0.402170,-1.623486,-0.739405,-0.864920,-0.826259,0.085565,-0.553293,0.553293
1733,0.360434,-0.017825,1.338291,1.308772,1.010307,1.203170,0.918554,0.085565,-0.553293,0.553293
143278,2.117875,3.968676,1.567299,0.095424,0.083989,0.114701,1.463809,0.630273,1.807362,-1.807362
92288,-0.278635,-0.273263,-0.173162,-0.612362,-1.254026,1.420864,-1.262462,0.085565,-0.553293,0.553293


# Training 3 Ensemble Models

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": xgb.XGBRegressor(n_estimators=100, random_state=42),
    "GradientBoost": GradientBoostingRegressor(n_estimators=100, random_state=42)
}
for model_name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    print(f"{model_name} Cross-Validation R2 Scores: {scores}")
    print(f"{model_name} Mean Cross-Validation R2 Score: {np.mean(scores)}")
    model.fit(X_train, y_train)
    test_score = model.score(X_test, y_test)
    print(f"{model_name} Test R2 Score: {test_score}\n")


RandomForest Cross-Validation R2 Scores: [0.96285735 0.96339231 0.96289367 0.96307758 0.96316591]
RandomForest Mean Cross-Validation R2 Score: 0.963077364776986
RandomForest Test R2 Score: 0.9639351275656334

XGBoost Cross-Validation R2 Scores: [0.95769752 0.95873459 0.95845839 0.95810048 0.95822681]
XGBoost Mean Cross-Validation R2 Score: 0.9582435590507898
XGBoost Test R2 Score: 0.9585913679904483

GradientBoost Cross-Validation R2 Scores: [0.93858756 0.94098078 0.94116223 0.9384382  0.94034235]
GradientBoost Mean Cross-Validation R2 Score: 0.9399022226258904
GradientBoost Test R2 Score: 0.9404733584283288



# Training 3 Basic Models

In [ ]:
# Linear Regression

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
Linear_model = LinearRegression()
Linear_model.fit(X_train, y_train)
y_pred_test = Linear_model.predict(X_test)
Linear_test_r2 = r2_score(y_test, y_pred_test)
Linear_test_mae = mean_absolute_error(y_test, y_pred_test)


In [ ]:
# Decision Tree Regression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
Decision_Tree_model = DecisionTreeRegressor(random_state=42)
Decision_Tree_model.fit(X_train, y_train)
Decision_Tree_y_pred_test = Decision_Tree_model.predict(X_test)
Decision_Tree_test_r2 = r2_score(y_test, Decision_Tree_y_pred_test)
Decision_Tree_test_mae = mean_absolute_error(y_test, Decision_Tree_y_pred_test)




# Evaluation and Hypertuning

In [ ]:
# Using evaluation metrics to assess our basic models

#Decision Tree Regression
test_mae = mean_absolute_error(y_test, y_pred_test)
print("Decision Tree Regression without hyperparameters:")
print(f"Test R2 score: {Decision_Tree_test_r2}")
print(f"Test MAE: {Decision_Tree_test_mae}")

Decision Tree Regression without hyperparameters:
Test R2 score: 0.9261531874535888
Test MAE: 1.1210972553145404


In [ ]:
# Evaluation On Linear Regression

# Linear Regression
print("Linear Regression without hyperparameters:")
print(f"Test R2 score: {Linear_test_r2}")
print(f"Test MAE: {Linear_test_mae}")

Linear Regression without hyperparameters:
Test R2 score: 0.854480376034291
Test MAE: 2.086535212010037


In [ ]:
# Using evaluation metrics to assess our ensemble models
for model_name, model in models.items():
    model.fit(X_train, y_train)

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    test_r2 = model.score(X_test, y_test)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)

    print(f"{model_name} Train MAE: {train_mae}")
    print(f"{model_name} Test MAE: {test_mae}")
    print(f"{model_name} Test R2 Score: {test_r2}\n")

RandomForest Train MAE: 0.3311688610160077
RandomForest Test MAE: 0.8957396397099959
RandomForest Test R2 Score: 0.9639351275656334

XGBoost Train MAE: 0.9926571382325424
XGBoost Test MAE: 1.0548601499615866
XGBoost Test R2 Score: 0.9585913679904483

GradientBoost Train MAE: 1.2937635247340538
GradientBoost Test MAE: 1.294253430253454
GradientBoost Test R2 Score: 0.9404733584283288



In [ ]:
# Hypertuning My Ensemble Model Of Choice

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgb_model = models["XGBoost"]

param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
}

random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_dist,
    n_iter=12,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)

print(f"Best parameters for XGBoost: {random_search.best_params_}")
print(f"Best cross-validation R2 score for XGBoost: {random_search.best_score_}")

best_model = random_search.best_estimator_

best_model.fit(X_train, y_train)
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)



In [ ]:
# Evaluating my hypertuned model

In [ ]:
print(f"XGBoost Train R2 Score: {train_r2}")
print(f"XGBoost Test R2 Score: {test_r2}")
print(f"XGBoost Train MAE: {train_mae}")
print(f"XGBoost Test MAE: {test_mae}")

# Loading and Preparing My Test Dataset

In [ ]:
players_data_path = '/content/drive/My Drive/players_22.csv'
players_data = pd.read_csv(players_data_path)
players_data.head()

In [ ]:
# Dropping Columns with Low Correlation in The Testing Dataset
players_data = drop_low_correlation_columns(players_data, 'overall')


In [ ]:
# Performing One Hot Encoding On The Preferred Foot Column
players_data = one_hot_encode_preferred_foot(players_data)

In [ ]:
# Dropping The Columns With Null Values greater than 30 percent of the total column
players_data = drop_high_null_columns(players_data, threshold=0.3)

In [ ]:
# Dropping All Categorical Columns From My DataSet
players_data = drop_categorical_columns(players_data)


In [ ]:
# Using median imputing to fill the null results with the median of the columns
players_data = median_imputing(players_data)


In [ ]:
# Scaling the independent variables in my testing data set
scaled_df, scaler = scale_independent_variables(players_data, 'overall')
players_data = scaled_df

In [ ]:
players_data

In [ ]:
# Making sure that only the columns available in the training data set are available in the testing data set
common_columns = result.columns
filtered_players_data = players_data[common_columns]

In [ ]:
# Testing my model against unseen data

In [ ]:
X_test.info()

In [ ]:
# Separating features from labels
X_test, y_test = filtered_players_data, filtered_players_data['overall']
X_test = X_test.drop('overall', axis=1)

# Make predictions
y_pred = best_model.predict(X_test)

# Evaluate the model's performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared Score: {r2}")

In [ ]:
# Using Feature Importance To Determine which Features To Put On My Webapp

In [ ]:
feature_importances = best_model.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()

top_features = feature_importance_df.head(10)['Feature']
print("Top Features:")
print(top_features)


In [ ]:
# Downloading Scalar Used On Training and Test Data To Be Applied Onto User's Input
with open('scalernew.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)